In [4]:
import numpy as np
import numba as nb
import scipy as sp
import math

In [9]:
@nb.njit()
def ufunc(x,nmom=5):
    def comb(n,k):
        if k > n:
            return 0
        return math.gamma(n+1) / (math.gamma(k+1) * math.gamma(n-k+1))
    
    x = np.asarray(x)
    n = x.shape[0]
    x.sort()

    if nmom >= 2:
        comb1 = np.arange(n)
        if nmom >= 3:
            comb3 = np.array([comb(i, 2) for i in np.arange(n)])
            if nmom >= 4:
                comb5 = np.array([comb(i, 3) for i in np.arange(n)])
                if nmom >= 5:
                    comb7 = np.array([comb(i, 4) for i in np.arange(n)])   

    # First L-moment

    if nmom == 1:
        l1 = np.sum(x,axis=0) / comb(n, 1)
        return l1
    
    # Second L-moment
        
    coefl2 = 0.5 / comb(n,2)
    sum_xtrans = np.zeros_like(x[0])
    for i in np.arange(n):
        sum_xtrans += (comb1[i] - comb1[n - i - 1]) * x[i]
    l2 = coefl2 * sum_xtrans

    if nmom == 2:
        return l2

    # Third L-moment

    if nmom == 3:
        coefl3 = 1.0 / 3.0 / comb(n, 3)
        sum_xtrans = np.zeros_like(x[0])
        for i in np.arange(n):
            sum_xtrans += (
                comb3[i] - 2 * comb1[i] * comb1[n - i - 1] + comb3[n - i - 1]
            ) * x[i]


        l3 = coefl3 * sum_xtrans / l2
        return l3

    # Fourth L-moment

    if nmom == 4:
        coefl4 = 0.25 / comb(n, 4)

        sum_xtrans = np.zeros_like(x[0])
        for i in np.arange(n):
            sum_xtrans += (
                comb5[i]
                - 3 * comb3[i] * comb1[n - i - 1]
                + 3 * comb1[i] * comb3[n - i - 1]
                - comb5[n - i - 1]
            ) * x[i]


        l4 = coefl4 * sum_xtrans / l2
        return l4

    # Fifth L-moment

    if nmom == 5:
        coefl5 = 0.2 / comb(n, 5)

        sum_xtrans = np.zeros_like(x[0])
        for i in np.arange(n):
            sum_xtrans += (
                comb7[i]
                - 4 * comb5[i] * comb1[n - i - 1]
                + 6 * comb3[i] * comb3[n - i - 1]
                - 4 * comb1[i] * comb5[n - i - 1]
                + comb7[n - i - 1]
            ) * x[i]

        l5 = coefl5 * sum_xtrans / l2

        return l5

In [7]:
testdata = np.random.rand(100,100,100)

In [12]:
def get_lmoments(x, nmom=5) -> np.ndarray:
    try:
        x = np.asarray(x, dtype=np.float64)
        n = x.shape[0]
        x.sort(axis=0)
    except ValueError:
        raise ValueError("Input data to estimate L-moments must be numeric.")

    if nmom <= 0 or nmom > 5:
        raise ValueError("Invalid number of sample L-moments")

    if n < nmom:
        raise ValueError("Insufficient length of data for specified nmoments")

    # First L-moment

    l1 = np.sum(x,axis=0) / sp.special.comb(n, 1, exact=True)

    if nmom == 1:
        return l1

    # Second L-moment

    comb1 = range(n)
    coefl2 = 0.5 / sp.special.comb(n, 2, exact=True)
    sum_xtrans = sum([(comb1[i] - comb1[n - i - 1]) * x[i] for i in range(n)])
    l2 = coefl2 * sum_xtrans

    if nmom == 2:
        return np.array([l1, l2])

    # Third L-moment

    comb3 = [sp.special.comb(i, 2, exact=True) for i in range(n)]
    coefl3 = 1.0 / 3.0 / sp.special.comb(n, 3, exact=True)
    sum_xtrans = sum(
        [
            (comb3[i] - 2 * comb1[i] * comb1[n - i - 1] + comb3[n - i - 1]) * x[i]
            for i in range(n)
        ]
    )
    l3 = coefl3 * sum_xtrans / l2

    if nmom == 3:
        return np.array([l1, l2, l3])

    # Fourth L-moment

    comb5 = [sp.special.comb(i, 3, exact=True) for i in range(n)]
    coefl4 = 0.25 / sp.special.comb(n, 4, exact=True)
    sum_xtrans = sum(
        [
            (
                comb5[i]
                - 3 * comb3[i] * comb1[n - i - 1]
                + 3 * comb1[i] * comb3[n - i - 1]
                - comb5[n - i - 1]
            )
            * x[i]
            for i in range(n)
        ]
    )
    l4 = coefl4 * sum_xtrans / l2

    if nmom == 4:
        return np.array([l1, l2, l3, l4])

    # Fifth L-moment

    comb7 = [sp.special.comb(i, 4, exact=True) for i in range(n)]
    coefl5 = 0.2 / sp.special.comb(n, 5, exact=True)
    sum_xtrans = sum(
        [
            (
                comb7[i]
                - 4 * comb5[i] * comb1[n - i - 1]
                + 6 * comb3[i] * comb3[n - i - 1]
                - 4 * comb1[i] * comb5[n - i - 1]
                + comb7[n - i - 1]
            )
            * x[i]
            for i in range(n)
        ]
    )
    l5 = coefl5 * sum_xtrans / l2

    return np.array([l1, l2, l3, l4, l5])

In [15]:
%timeit ufunc(testdata,nmom=5)

19.3 ms ± 1.49 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
%timeit get_lmoments(testdata,nmom=5)

36.5 ms ± 8.45 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
